<h2>Rice Coding Compression<h2/>

<p>We need to be certain modules for the application is functioning especially for the wav/byte/bit manipulation.</p>

In [3]:
!pip install bitstring

import math
import wave

from bitstring import BitStream, BitArray

     |████████████████████████████████| 45 kB 14.9 MB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python3 -m pip install --upgrade pip' command.


<h3>Encode</h3>

In [4]:
# Unary function for encoding
def unary(t):
    y=[];
    for i in range(t):
        y.append('1')
    y.append('0')
    return ''.join(y)

# Rice encoding algorithm slice
def r_enc(S, K):

    # find M
    M = 2 ** K

    # For S to be encoded, find:
    # Quotient q = int(S/M)
    q = int(S//M)

    # Remainder r = SmoduloM
    r = S%M
    
    # Quotient code is q in unary
    q_un = unary(q)

    # Remainder code is r in binary using K bits
    # Function to return x in n-bit binary
    getbinary = lambda x, n: format(x, 'b').zfill(n)
    r_bin = getbinary(r, K)

    # Codeword format <quotient code><remainder code>
    codeword = q_un + r_bin
    
    return codeword

<p>This reads our wav file and returns a byte object</p>

In [14]:
# Getting binary data from the wav file 
w = wave.open('Ex2_files/Sound1.wav', 'rb')
byte_data = w.readframes(w.getnframes())
w.close()

<p>We parse this byte object and encoding each byte into bits as per the rice coding algorithm. The results are being stored as a list bit each a result of r_enc</p>

In [15]:
# Bytes to bitstrings
bits = []
for byte in byte_data:
    bits.append(r_enc(byte, 4))

<p>We create an ex2 file to store our bits. We use the join method to store one large bit string separated by a space so we can split it later for decoding</p>

In [16]:
# Encode and write to ex2 file
with open("Ex2_files/Sound1_Enc.ex2", "w") as f:
    # Writing data to a file
    f.write(' '.join(bits))

<p>Encoding of the wav file and saving it</p>

<h3>Decoding</h3>

<p>Decodeing is done here using the rice algorithm.</p>

In [11]:
# Rice decoding algorithm snippet
def r_dec(codeword, K):
    cdw = list(codeword)
    
    # q by counting 1's before first 0
    q = 0
    
    for i in cdw:
        if i == '1':
            q += 1
        else:
            break
    
    # r reading next K bits as binary value
    r = ''.join(cdw[q:])
    r_int = int(r, 2)
    
    M = 2 ** K
    
    # S, encoded number, as q x M + r
    S = q * M + r_int
    
    return S

<p>Parsing the ex2 file using the split method and append each bit block to an array</p>

In [13]:
bitsagain = []
    
with open("Ex2_files/Sound1_Enc.ex2", "r") as fd:
    bitsagain = fd.read().split(' ')

<p>Decode_blocklist takes this array and a K value for the rice algorithm and returns a list of our original bytes</p>

In [17]:
def decode_blocklist(blocklist, K):
    nums = []
        
    for block in blocklist:
        nums.append(r_dec(block, K))

    return nums

<p>Store our decoded list as a variable for usage later on</p>

In [18]:
# Decoded list of integers from bitstring
decbits = decode_blocklist(bitsagain, 4)

<p>Conversion of this list back to a bytes object of which we check if it is true.</p>

In [19]:
# conversion of int list to bytes
bytesagain = bytes(decbits)
type(bytesagain)

bytes

<p>Confirmation on the equivalence of the original bytes object from our wav file to the output of the r_dec -> bytes() process</p>

In [20]:
# Equivalency check of files
bytesagain == byte_data

True

<p>We then write the result back to a wav file tagged with 'Dec'.</p>

In [21]:
with open('Ex2_files/Sound1_Enc_Dec.wav', mode='bx') as f:
    f.write(bytesagain)

<p>Observing both wav files in the directory they are both identical in size.</p>

<h3>Finding the Optimal K</h3>

<p>We try to find the optimal K here</p>

<p>We see K = 7 can represent 255 in  bits, and 1 in 8. Let us select 10 as our K.</p>

In [22]:
len(r_enc(255, 10))

11

In [23]:
len(r_enc(1, 10))

11

<p>Open our Sound2.wav file as a byte object for compressing.</p>

In [25]:
# getting binary data from wav file 
w = wave.open('Ex2_files/Sound2.wav', 'rb')
byte_data2 = w.readframes(w.getnframes())
w.close()

<p>Convert our Sound1.wav bytes using r_enc.</p>

In [27]:
# bytes to bitstrings
bits2 = []
for byte in byte_data:
    bits2.append(r_enc(byte, 10))

<p>Convert our Sound2.wav bytes using r_enc.</p>

In [28]:
# bytes to bitstrings
bits3 = []
for byte in byte_data2:
    bits3.append(r_enc(byte, 10))

<p>Write results of Sound1.wav encode to file.</p>

In [30]:
# Encode and write to ex2 file
with open("Ex2_files/Sound1_K7_Enc.ex2", "w") as f:
    # Writing data to a file
    f.write(' '.join(bits2))

<p>Write results of Sound2.wav encode to file.</p>

In [33]:
# Encode and write to ex2 file
with open("Ex2_files/Sound2_K7_Enc.ex2", "w") as f:
    # Writing data to a file
    f.write(' '.join(bits3))

<p>The compression results will be documented in the report</p>